<a href="https://colab.research.google.com/github/Suryareddy180/Style-Transfer-/blob/main/STYLETRANSFER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**STYLE TRANSFER USING  PRE-TRAINED MODEL**

In [2]:
!pip install tensorflow_hub opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 109.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 118.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully

In [5]:
!pip install moviepy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 60.3 MB/s eta 0:00:00


In [2]:
import tensorflow_hub as hub
import tensorflow as tf
import cv2
import numpy as np
from moviepy.editor import VideoFileClip, AudioFileClip

# Load TFHub style transfer model
model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

# Load and preprocess image
def load_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = img[tf.newaxis, :]
    return img

# Apply style transfer
def style_transfer_frame(content_image, style_image):
    stylized_image = model(tf.constant(content_image), tf.constant(style_image))[0]
    return stylized_image.numpy()

# Advanced sharpening (edge-preserving)
def sharpen_image(img):
    # Unsharp masking technique
    gaussian = cv2.GaussianBlur(img, (0, 0), 3)
    sharpened = cv2.addWeighted(img, 1.5, gaussian, -0.5, 0)
    return sharpened

# (Optional) Apply Super-Resolution using OpenCV's DNN
def super_resolution(img):
    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    sr.readModel("EDSR_x3.pb")  # You can download from OpenCV GitHub
    sr.setModel("edsr", 3)
    return sr.upsample(img)

# Main video processing function
def style_transfer_video(input_video_path, style_image_path, output_video_path):
    style_image = load_image(style_image_path)

    cap = cv2.VideoCapture(input_video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    temp_output_path = 'temp_stylized_video.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(temp_output_path, fourcc, fps, (width, height))

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Upscale to 512x512 before stylizing
        content_image = cv2.resize(frame, (512, 512))
        content_image = tf.image.convert_image_dtype(content_image, tf.float32)
        content_image = content_image[tf.newaxis, :]

        # Apply style transfer
        stylized = style_transfer_frame(content_image, style_image)

        # Resize back to original resolution
        stylized_resized = tf.image.resize(stylized, (height, width))
        stylized_np = (stylized_resized.numpy().squeeze() * 255).astype(np.uint8)
        stylized_bgr = cv2.cvtColor(stylized_np, cv2.COLOR_RGB2BGR)

        # Apply sharpening
        sharp_frame = sharpen_image(stylized_bgr)

        # Optional: Apply super-resolution
        # sharp_frame = super_resolution(sharp_frame)

        out.write(sharp_frame)
        frame_count += 1
        print(f'Processed frame {frame_count}/{total_frames}')

    cap.release()
    out.release()

    # Combine with original audio
    video_clip = VideoFileClip(temp_output_path)
    audio = AudioFileClip(input_video_path)
    final_video = video_clip.set_audio(audio)

    # Export in high bitrate for better visual quality
    final_video.write_videofile(output_video_path, codec="libx264", audio_codec="aac", bitrate="5000k")

# Example usage
input_video_path = '/content/stock3.mp4'
style_image_path = '/content/scene9.jpg'
output_video_path = '/content/output_super_high_quality.mp4'

style_transfer_video(input_video_path, style_image_path, output_video_path)


Processed frame 1/621
Processed frame 2/621
Processed frame 3/621
Processed frame 4/621
Processed frame 5/621
Processed frame 6/621
Processed frame 7/621
Processed frame 8/621
Processed frame 9/621
Processed frame 10/621
Processed frame 11/621
Processed frame 12/621
Processed frame 13/621
Processed frame 14/621
Processed frame 15/621
Processed frame 16/621
Processed frame 17/621
Processed frame 18/621
Processed frame 19/621
Processed frame 20/621
Processed frame 21/621
Processed frame 22/621
Processed frame 23/621
Processed frame 24/621
Processed frame 25/621
Processed frame 26/621
Processed frame 27/621
Processed frame 28/621
Processed frame 29/621
Processed frame 30/621
Processed frame 31/621
Processed frame 32/621
Processed frame 33/621
Processed frame 34/621
Processed frame 35/621
Processed frame 36/621
Processed frame 37/621
Processed frame 38/621
Processed frame 39/621
Processed frame 40/621
Processed frame 41/621
Processed frame 42/621
Processed frame 43/621
Processed frame 44/6

MoviePy - Done.
Moviepy - Writing video /content/output_super_high_quality.mp4



Moviepy - Done !
Moviepy - video ready /content/output_super_high_quality.mp4
